# Test to see if VAE can produce a good model

In [1]:
import tensorflow as tf
import tensorflow_probability as tfp
import keras
from keras import layers
import numpy as np
import matplotlib.pyplot as plt
import gym

In [2]:
from vae_recurrent import VAE, create_decoder, create_encoder
from util import random_observation_sequence, transform_observations

In [3]:
env = gym.make('MountainCarContinuous-v0')
env.action_space.seed(42)

observation_max = np.array([0.6, 0.07])
observation_min = np.array([-1.2, -0.07])

all_observations = []
all_observations_scaled = []
num_episodes = 10

obs_stddev = [0.05, 0.05]
# obs_stddev = [0, 0]

for i in range(num_episodes):
    o, a, r = random_observation_sequence(env, 1000, epsilon=0.1)
    # o_scaled = transform_observations(o, observation_max, observation_min, [0.05, 0.05])
    o_scaled = transform_observations(o, observation_max, observation_min, obs_stddev)
    o_scaled

    o = transform_observations(o, observation_max, observation_min, [0, 0])

    all_observations_scaled.append(o_scaled)
    all_observations.append(o)

all_observations = np.vstack(all_observations)
all_observations_scaled = np.vstack(all_observations_scaled)
all_observations.shape

(5310, 2)

In [4]:
enc = create_encoder(2, 2, [20])
dec = create_decoder(2, 2, [20])

vae = VAE(enc, dec, [0, 0], [0.3, 0.3], llik_scaling=1, kl_scaling=1, recon_stddev=0.05)
vae.compile(optimizer=tf.keras.optimizers.Adam())

Metal device set to: Apple M1 Pro


2022-08-11 17:28:08.015893: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-08-11 17:28:08.016127: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [5]:
vae.fit(all_observations_scaled, epochs=20, batch_size=64)

Epoch 1/20
Instructions for updating:
`scale_identity_multiplier` is deprecated; please combine it into `scale_diag` directly instead.


2022-08-11 17:28:09.294097: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


 1/83 [..............................] - ETA: 36s - loss: 127.6805 - reconstruction_loss: 118.8154 - kl_loss: 8.8652

2022-08-11 17:28:09.554095: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


83/83 [==============================] - 1s 9ms/step - loss: 88.8093 - reconstruction_loss: 64.1167 - kl_loss: 7.1859
Epoch 2/20
83/83 [==============================] - 1s 8ms/step - loss: 51.2224 - reconstruction_loss: 45.3210 - kl_loss: 5.5905
Epoch 3/20
83/83 [==============================] - 1s 8ms/step - loss: 45.3774 - reconstruction_loss: 39.1672 - kl_loss: 4.6245
Epoch 4/20
83/83 [==============================] - 1s 8ms/step - loss: 34.9713 - reconstruction_loss: 25.9926 - kl_loss: 4.6857
Epoch 5/20
83/83 [==============================] - 1s 7ms/step - loss: 20.1814 - reconstruction_loss: 11.6113 - kl_loss: 5.7716
Epoch 6/20
83/83 [==============================] - 1s 8ms/step - loss: 11.7913 - reconstruction_loss: 4.7133 - kl_loss: 6.1869
Epoch 7/20
83/83 [==============================] - 1s 8ms/step - loss: 9.2654 - reconstruction_loss: 3.0579 - kl_loss: 5.8366
Epoch 8/20
83/83 [==============================] - 1s 8ms/step - loss: 7.6671 - reconstruction_loss: 2.3183 - 

In [23]:
all_observations

array([[0.4186824 , 0.5       ],
       [0.41815374, 0.49320282],
       [0.41710026, 0.48645527],
       ...,
       [0.66083371, 0.28068669],
       [0.64255316, 0.26496433],
       [0.62306083, 0.24938429]])

In [24]:
all_observations_scaled

array([[0.44121964, 0.54672594],
       [0.35964572, 0.49859698],
       [0.42061976, 0.49774704],
       ...,
       [0.57533129, 0.22057353],
       [0.60133602, 0.27459678],
       [0.6011072 , 0.25962561]])

In [25]:
vae(all_observations_scaled)

<tf.Tensor: shape=(5410, 2), dtype=float32, numpy=
array([[0.28611946, 0.57185763],
       [0.3173399 , 0.5144284 ],
       [0.47711825, 0.50293446],
       ...,
       [0.54905176, 0.3013838 ],
       [0.5998849 , 0.26799646],
       [0.6170516 , 0.33103257]], dtype=float32)>

In [12]:
0.5*np.log(2*np.pi*(0.05**2))

-2.076793740349318

In [15]:
enc = create_encoder(2, 2, [20])
dec = create_decoder(2, 2, [20])
vae = VAE(enc, dec, [0, 0], [0.3, 0.3], llik_scaling=10000, kl_scaling=1)
vae.compile(optimizer=tf.keras.optimizers.Adam())
vae(all_observations_scaled)

<tf.Tensor: shape=(5843, 2), dtype=float32, numpy=
array([[0.53299767, 0.5704217 ],
       [0.6018372 , 0.44273585],
       [0.58923995, 0.44603327],
       ...,
       [0.50266194, 0.48137212],
       [0.45818692, 0.52328986],
       [0.44035968, 0.50200146]], dtype=float32)>

Random test to make sure posterior collapse is avoided

In [119]:
a = np.random.random((10, 2))
print(a)
vae.decoder(a)

[[0.12924743 0.30960184]
 [0.34146105 0.63325081]
 [0.2006226  0.23677983]
 [0.39715259 0.65883864]
 [0.33274673 0.92715291]
 [0.30385739 0.43389812]
 [0.20808629 0.38681099]
 [0.77297407 0.25049935]
 [0.63391301 0.06432549]
 [0.41720471 0.97503232]]


<tf.Tensor: shape=(10, 2), dtype=float32, numpy=
array([[0.2079564 , 0.3798874 ],
       [0.17498101, 0.44049573],
       [0.19405416, 0.3613656 ],
       [0.1657172 , 0.4435221 ],
       [0.18144417, 0.5038548 ],
       [0.17846933, 0.3992164 ],
       [0.19482864, 0.39318183],
       [0.10141916, 0.33977422],
       [0.11905786, 0.30723736],
       [0.16754702, 0.51014173]], dtype=float32)>

## MNIST

In [120]:
num_classes = 10
input_shape = (28, 28, 1)

# Load the data and split it between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

x_train = x_train/255
x_train_flat = x_train.reshape(x_train.shape[0], x_train.shape[1]**2)

In [121]:
x_train.max()

1.0

In [122]:
enc = create_encoder(10, 784, [256, 128])
dec = create_decoder(10, 784, [256, 128])

vae = VAE(enc, dec, [0]*10, [1]*10, llik_scaling=100, kl_scaling=1)
vae.compile(optimizer=tf.keras.optimizers.Adam())

In [123]:
vae.fit(x_train_flat, batch_size=64, epochs=20)

Epoch 1/20


2022-08-07 13:54:56.950320: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


938/938 [==============================] - 10s 10ms/step - loss: 32.1690 - reconstruction_loss: 26.0431 - kl_loss: 1.0763
Epoch 2/20
938/938 [==============================] - 9s 10ms/step - loss: 24.7607 - reconstruction_loss: 22.7322 - kl_loss: 1.7090
Epoch 3/20
938/938 [==============================] - 9s 9ms/step - loss: 23.9416 - reconstruction_loss: 21.5231 - kl_loss: 2.2955
Epoch 4/20
938/938 [==============================] - 9s 9ms/step - loss: 23.3899 - reconstruction_loss: 20.2792 - kl_loss: 2.9727
Epoch 5/20
196/938 [=====>........................] - ETA: 7s - loss: 23.0395 - reconstruction_loss: 19.8954 - kl_loss: 3.1952

KeyboardInterrupt: 

In [ ]:
out = vae(x_train_flat)
out = out.numpy().reshape(out.shape[0], 28, 28)
out.shape

In [ ]:
example = 16
plt.imshow(out[example])

In [ ]:
plt.imshow(x_train[example])

In [ ]:
z = np.random.normal([0]*10, scale=[1]*10, size=(1, 10))
fake_num = vae.decoder(z).numpy().reshape(28,28)
plt.imshow(fake_num)